In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [2]:
col = ['feature2','feature3','feature7','class']


# Data input

In [3]:
train_data = pd.read_csv("./data/train_dec08_task4_missing.csv")
sup_data = pd.read_csv("./data/train_dec10_task4_missing_supplement.csv")
test_data = pd.read_csv("./data/test_dec08_task4_missing_only_features.csv")

In [4]:
train_data = train_data[col].copy()
sup_data = sup_data[col].copy()
train_data = train_data.dropna()
sup_data = sup_data.dropna()

In [5]:
sup_data.iloc[1:10]

,feature2,feature3,feature7,class
1,20.176954,15.535818,8.334131,A
2,19.973465,15.704548,4.626235,A
3,7.418786,7.192065,7.834803,A
4,20.001505,17.789371,5.797457,A
5,10.191896,16.274527,7.492501,A
6,23.788850,16.358599,9.937226,A
7,19.316132,15.947508,2.176455,A
8,28.216307,19.733914,9.570840,A
9,10.354382,9.882882,9.736760,A


In [6]:
sup_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6106 entries, 0 to 6498
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   feature2  6106 non-null   float64
 1   feature3  6106 non-null   float64
 2   feature7  6106 non-null   float64
 3   class     6106 non-null   object 
dtypes: float64(3), object(1)
memory usage: 238.5+ KB


In [7]:
train_data['class'].value_counts()

C    2104
B    1663
A     839
D     828
Name: class, dtype: int64

In [8]:
train_data = pd.concat([train_data, sup_data])

In [9]:
#train_data = train_data.reset_index()

In [10]:
train_data['class'].value_counts()

C    4455
B    3544
A    1782
D    1759
Name: class, dtype: int64

# Imputator

# 效果不好 所以選擇把missing value drop掉

In [11]:
#impute = IterativeImputer(estimator=KNeighborsRegressor(n_neighbors = 3), random_state=0)
#impute.fit(train_data[['feature2','feature3','feature4']])
#features = impute.transform(train_data[['feature2','feature3','feature4']])

In [12]:
#features = pd.DataFrame(data = features, columns = ['feature2','feature3','feature4'])

In [13]:
#features

In [14]:
#train_data[['class']]

In [15]:

#train_data = pd.concat([features[['feature2','feature3','feature4']],train_data[['class']]], axis = 1)

In [16]:
#train_data

# Downsampling
取各1700個data

In [17]:
x = pd.DataFrame(data = None, columns= train_data.columns)

In [19]:
from sklearn.utils import resample
classifications = ['A','D','B','C']
np1 = np.array([])
for c in classifications:
  np1 = np.append(np1,(resample(train_data[train_data['class'] == c],
             replace=False,
             n_samples=1700,
             random_state=42)))
#np1 = np.append(np1,np.array(train_data[train_data['class'] == 'A']))
#np1 = np.append(np1,np.array(train_data[train_data['class'] == 'D']))
x = pd.DataFrame(data = np1.reshape(int(27200/4),4), columns= train_data.columns)
#x.append([train_data[train_data['class'] == 'A'],train_data[train_data['class'] == 'D']])


In [20]:
x

,feature2,feature3,feature7,class
0,16.816507,21.232858,5.683259,A
1,11.519243,15.446547,4.359459,A
2,10.314918,22.960186,4.591183,A
3,16.141004,21.075877,8.108052,A
4,6.931165,13.423816,1.12463,A
...,...,...,...,...
6795,13.332396,18.462377,1.242204,C
6796,16.320013,18.704451,6.787509,C
6797,13.366113,14.296811,9.239785,C
6798,13.344825,30.027203,1.191801,C


In [21]:
x['class'].value_counts()

A    1700
D    1700
B    1700
C    1700
Name: class, dtype: int64

In [22]:
X = x[col[:-1]]
y = x['class']

In [23]:
from sklearn import preprocessing
#zscore = preprocessing.StandardScaler()
std = preprocessing.StandardScaler()
X = std.fit_transform(X)
t = std.fit_transform(pd.concat([test_data[col[:-1]]], axis =1))

In [24]:
temp = pd.DataFrame(data = X, columns = col[:-1])

In [25]:
temp = pd.concat([temp,y],axis = 1)

In [26]:
temp

,feature2,feature3,feature7,class
0,-0.054726,0.898473,0.259151,A
1,-0.892207,-0.098390,-0.234447,A
2,-1.082607,1.196056,-0.148046,A
3,-0.161521,0.871429,1.163271,A
4,-1.617568,-0.446865,-1.440602,A
...,...,...,...,...
6795,-0.605553,0.421176,-1.396763,C
6796,-0.133220,0.462880,0.670887,C
6797,-0.600223,-0.296466,1.585255,C
6798,-0.603588,2.413559,-1.415556,C


# Outlier detection
# 加了反而表現不好

In [27]:
#def outliers(df, ft):
#  ls = []
#  for c in ['A','B','C','D']:
#    Q1 = df[ft].loc[df['class'] == c].quantile(0.25)
#    Q3 = df[ft].loc[df['class'] == c].quantile(0.75)
 #   IQR = Q3 -Q1
 #   lower_bound = Q1 -1.5 * IQR
 #   upper_bound = Q3 + 1.5 * IQR
 #   
 #   ls.extend(df.index[(df[ft] < lower_bound) | (df[ft] > upper_bound)])
 #   
 # return ls

In [28]:
#index_list = []
#for feature in ['feature2', 'feature3', 'feature4']:
#    index_list.extend(outliers(temp, feature))
#  

In [29]:
#def remove(df, ls):
#    ls = sorted(set(ls))
#    df = df.drop(ls)
#    return df

In [30]:
#temp = remove(temp,index_list)

In [31]:
#temp

In [32]:
#temp[['class']]

# Training

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(temp[col[:-1]], temp[['class']], test_size=0.15, random_state=42)

In [34]:
from sklearn.linear_model import LogisticRegression
lgr = LogisticRegression(random_state = 42)
lgr.fit(X_train, y_train)

/Users/jonner/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression(random_state=42)

In [35]:
from sklearn.model_selection import cross_validate
scores = cross_validate(lgr,X_val,y_val,cv=5,scoring='accuracy')
print(scores.get('test_score'))
print(np.sum(scores.get('test_score'))/5)

[0.38235294 0.3627451  0.39215686 0.34803922 0.36764706]
0.37058823529411766


/Users/jonner/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/jonner/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/jonner/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/jonner/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was e

In [36]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=10, min_samples_split=10)
clf.fit(X_train, y_train)

/var/folders/8s/2rgq66zj4q7bpsf8g93fj0bh0000gn/T/ipykernel_8396/2217275760.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  clf.fit(X_train, y_train)


RandomForestClassifier(min_samples_split=10, n_estimators=10)

In [37]:
scores = cross_validate(clf,X_val,y_val,cv=5,scoring='accuracy')
scores

/Users/jonner/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/jonner/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/jonner/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/jonner/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation

{'fit_time': array([0.03474593, 0.02243495, 0.01534891, 0.01255894, 0.01216793]),
 'score_time': array([0.00446105, 0.00216293, 0.00168204, 0.00144315, 0.00143099]),
 'test_score': array([0.33823529, 0.45098039, 0.42156863, 0.34313725, 0.32843137])}

# Deep Learning

In [37]:
#from tensorflow.keras.utils import to_categorical
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense
#from tensorflow.keras.optimizers import SGD

In [38]:
#def toCategory(lst):
#  t = []
#  for i in range(0,len(lst)):
#      k = lst
#      if(k[i] == 'A'):
#          t.append(0)
#      elif(k[i] == 'B'):
#          t.append(1)
#      elif(k[i] == 'C'):
#         t.append(2)
#      elif(k[i] == 'D'):
#         t.append(3)
#  t = to_categorical(t, 4)
#  return t;
#y = toCategory(np.array(temp[['class']]))

IndentationError: unexpected indent (3370263089.py, line 2)

In [ ]:
#y

In [ ]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(temp[col[:-1]], y, test_size=0.15, random_state = 42)

In [ ]:
#model = Sequential()
#model.add(Dense(100, input_dim=3, activation='relu'))
#model.add(Dense(50, activation='relu'))
#
#model.add(Dense(4, activation='softmax'))
#
#model.compile(loss='categorical_crossentropy', optimizer='adam', 
#              metrics=['accuracy'])

In [ ]:
#model.fit(X_train, y_train, batch_size=33, epochs=20, validation_data=(X_test, y_test))

# Output our answer

In [39]:
#要輸出傳統機器學習模型請使用此cell
test_data = (pd.DataFrame(t, columns = col[:-1])).fillna(0)
#test_data = impute.transform(test_data)
#test_data = std.transform(test_data)
answer =lgr.predict(test_data)

In [40]:
#要輸出深度學習模型請使用此cell
#test_data = pd.DataFrame(t, columns = col[:-1]).fillna(0)
#answer = np.argmax(model.predict(test_data),axis = -1)

In [42]:
#要輸出深度學習模型請使用此cell
#ans = []
#for a in answer:
#  if(a == 0):
#    ans.append("A")
#  elif(a == 1):
#    ans.append("B")
#  elif(a == 2):
#    ans.append("C")
#  elif(a == 3):
#     ans.append("D")

In [43]:
id = np.linspace(1,1600,1600,dtype = "int")
a = np.concatenate((id,answer))#深度學習請使用ans 傳統機器學習請用answer
a = a.reshape(2,1600)
a = np.transpose(a, axes=None)
df = pd.DataFrame(a, columns = ['Id','Category'], index = False)
df.to_csv('./answer4.csv', columns=['Id','Category'])